In [1]:
import pandas as pd
import pandas_gbq
from google.cloud import bigquery
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score
from deepctr.models import DeepFM,DIN
from deepctr.feature_column import SparseFeat, VarLenSparseFeat, DenseFeat, get_feature_names
import numpy as np
import tensorflow as tf
import random

In [2]:
import pickle
def save_variable(v,filename):
    f=open(filename,'wb')
    pickle.dump(v,f)
    f.close()
    return filename
def load_variable(filename):
    f=open(filename,'rb')
    r=pickle.load(f)
    f.close()
    return r


### start training(use 4/1-5/1 model train5/2 data)

In [32]:
df_uid_5 = load_variable('/home/jovyan/dataset/may_df_uid')
df_ven_guid_5 = load_variable('/home/jovyan/dataset/may_df_ven_guid')
df_uid52 = df_uid_5[df_uid_5.month==52]
df_ven_guid52 = df_ven_guid_5[df_ven_guid_5.month==52]

In [41]:
len(df_uid52)

82333

In [33]:
df_temp = pd.concat([df_uid52[['month','gid_encode']],df_ven_guid52[['month','gid_encode']]],ignore_index=True)
under = 0
freq_list = []
freq = df_temp.gid_encode.value_counts()
for i in freq.values:
    under += np.log10(i+1)
for i in freq.values:
    freq_list.append(np.log10(i+1)/under)

In [34]:
import random
def create_set_byuid(df):
    t_set = []
    neg_list = random.choices(freq.index,weights = freq_list, k=1000000)
    for userid, hist in df.groupby('uid'):
        pos_list = hist['gid_encode'].tolist()
        if len(pos_list) > 101:
            pos_list = pos_list[-101:]
            print(userid)
        def gen_neg():
            count = 0
            neg = pos_list[0]
            while neg in pos_list:
                if count == 100:
                    count = 0
                neg = random.choices(neg_list, k=1)[0]
                count += 1
            return neg
        neg_list2 = [gen_neg() for i in range(len(pos_list))]
        for i in range(1, len(pos_list)):
            hist = pos_list[:i] 
            t_set.append((userid, hist, pos_list[i], 1))
            t_set.append((userid, hist, neg_list2[i], 0))
    return t_set
def create_set_byvenguid(df):
    t_set = []
    neg_list = random.choices(freq.index,weights = freq_list, k=1000000)
    for userid, hist in df.groupby('ven_guid'):
        pos_list = hist['gid_encode'].tolist()
        if len(pos_list) > 101:
            pos_list = pos_list[-101:]
            print(userid)
        def gen_neg():
            count = 0
            neg = pos_list[0]
            while neg in pos_list:
                if count == 100:
                    count = 0
                neg = random.choices(neg_list, k=1)[0]
                count += 1
            return neg
        neg_list2 = [gen_neg() for i in range(len(pos_list))]
        for i in range(1, len(pos_list)):
            hist = pos_list[:i] 
            t_set.append((userid, hist, pos_list[i], 1))
            t_set.append((userid, hist, neg_list2[i], 0))
    return t_set

df_train_uid = create_set_byuid(df_uid52)
df_train_venguid = create_set_byvenguid(df_ven_guid52)
train_set502 = df_train_venguid + df_train_uid
np.random.shuffle(train_set502)
len(df_train_venguid),len(df_train_uid)

144304
179401
574791


(214282, 131154)

In [36]:
def create_array(data):
    g_id = []
    hist_g_id = []
    click = []
    seq_length = []

    for index, value in enumerate(data):
        hist_g_id.append(value[1])
        seq_length.append(len(value[1]))
        g_id.append(value[2])
        click.append(value[3])  
    for i in range(len(hist_g_id)):
        if len(hist_g_id[i]) != 100:
            hist_g_id[i] += (100-len(hist_g_id[i]))*[0]

    g_id = np.array(g_id)
    hist_g_id = np.array(hist_g_id)
    click = np.array(click)
    seq_length = np.array(seq_length)
    return g_id, hist_g_id, click, seq_length
g_id_train502, hist_g_id_train502, click_train502, seq_length_train502 = create_array(train_set502)

In [37]:
behavior_feature_list = ["g_id"]
feature_columns = [SparseFeat('g_id', 1870599, embedding_dim=8)]
feature_columns += [VarLenSparseFeat(SparseFeat('hist_g_id',vocabulary_size=1870599, embedding_dim=8, embedding_name='g_id'), maxlen = 100, length_name='seq_length')]

x = {'g_id': g_id_train502, 
    'hist_g_id': hist_g_id_train502, 
    'seq_length': seq_length_train502}
y = click_train502

In [69]:
model = DIN(feature_columns, behavior_feature_list)
model.compile('SGD', 'binary_crossentropy',metrics=['accuracy','AUC'])
latest = tf.train.latest_checkpoint("/home/jovyan/dataset/checkpoint_folder/model_401_501")
model.load_weights(latest)

checkpoint_path = "/home/jovyan/dataset/checkpoint_folder/model_401_502/rakuten_0401_0502.ckpt"
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, save_weights_only=True,verbose=1)
history401_502 = model.fit(x, y, batch_size=128, verbose=1, epochs=10, callbacks=[cp_callback])

Epoch 1/10
2699/2699 [==============================] - 690s 254ms/step - loss: 0.5652 - accuracy: 0.7065 - auc: 0.7786

Epoch 00001: saving model to /home/jovyan/dataset/checkpoint_folder/rakuten_0401_0502.ckpt
Epoch 2/10
2699/2699 [==============================] - 681s 252ms/step - loss: 0.4714 - accuracy: 0.7754 - auc: 0.8542

Epoch 00002: saving model to /home/jovyan/dataset/checkpoint_folder/rakuten_0401_0502.ckpt
Epoch 3/10
2699/2699 [==============================] - 690s 256ms/step - loss: 0.3885 - accuracy: 0.8283 - auc: 0.9039

Epoch 00003: saving model to /home/jovyan/dataset/checkpoint_folder/rakuten_0401_0502.ckpt
Epoch 4/10
2699/2699 [==============================] - 696s 258ms/step - loss: 0.3190 - accuracy: 0.8681 - auc: 0.9360

Epoch 00004: saving model to /home/jovyan/dataset/checkpoint_folder/rakuten_0401_0502.ckpt
Epoch 5/10
2699/2699 [==============================] - 695s 257ms/step - loss: 0.2684 - accuracy: 0.8942 - auc: 0.9549

Epoch 00005: saving model to /h

### 15個epoch

In [40]:
model = DIN(feature_columns, behavior_feature_list)
model.compile('SGD', 'binary_crossentropy',metrics=['accuracy','AUC'])
latest = tf.train.latest_checkpoint("/home/jovyan/dataset/checkpoint_folder/model_401_502")
model.load_weights(latest)

checkpoint_path =  "/home/jovyan/dataset/15epoch/model401_502/401_502.ckpt"
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, save_weights_only=True,verbose=1)
history15epoch = model.fit(x, y, batch_size=128, verbose=1, epochs=5, callbacks=[cp_callback])

Epoch 1/5
2699/2699 [==============================] - 693s 255ms/step - loss: 0.3597 - accuracy: 0.8539 - auc: 0.9201

Epoch 00001: saving model to /home/jovyan/dataset/15epoch/model401_502/401_502.ckpt
Epoch 2/5
2699/2699 [==============================] - 700s 259ms/step - loss: 0.2562 - accuracy: 0.9042 - auc: 0.9593

Epoch 00002: saving model to /home/jovyan/dataset/15epoch/model401_502/401_502.ckpt
Epoch 3/5
2699/2699 [==============================] - 711s 263ms/step - loss: 0.2012 - accuracy: 0.9293 - auc: 0.9745

Epoch 00003: saving model to /home/jovyan/dataset/15epoch/model401_502/401_502.ckpt
Epoch 4/5
2699/2699 [==============================] - 694s 257ms/step - loss: 0.1665 - accuracy: 0.9441 - auc: 0.9822

Epoch 00004: saving model to /home/jovyan/dataset/15epoch/model401_502/401_502.ckpt
Epoch 5/5
2699/2699 [==============================] - 721s 267ms/step - loss: 0.1401 - accuracy: 0.9555 - auc: 0.9870

Epoch 00005: saving model to /home/jovyan/dataset/15epoch/model4

### 20 epoch

In [43]:
model = DIN(feature_columns, behavior_feature_list)
model.compile('SGD', 'binary_crossentropy',metrics=['accuracy','AUC'])
latest = tf.train.latest_checkpoint("/home/jovyan/dataset/15epoch/model401_502")
model.load_weights(latest)

checkpoint_path = "/home/jovyan/dataset/20epoch/model401_502/401_502.ckpt"
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, save_weights_only=True,verbose=1)
histtory5epoch = model.fit(x, y, batch_size=128, verbose=1, epochs=5, callbacks=[cp_callback])

Epoch 1/5
2699/2699 [==============================] - 729s 269ms/step - loss: 0.1257 - accuracy: 0.9614 - auc: 0.9892

Epoch 00001: saving model to /home/jovyan/dataset/20epoch/model401_502/401_502.ckpt
Epoch 2/5
2699/2699 [==============================] - 735s 272ms/step - loss: 0.1135 - accuracy: 0.9661 - auc: 0.9908

Epoch 00002: saving model to /home/jovyan/dataset/20epoch/model401_502/401_502.ckpt
Epoch 3/5
2699/2699 [==============================] - 738s 274ms/step - loss: 0.1060 - accuracy: 0.9691 - auc: 0.9916

Epoch 00003: saving model to /home/jovyan/dataset/20epoch/model401_502/401_502.ckpt
Epoch 4/5
2699/2699 [==============================] - 739s 274ms/step - loss: 0.0954 - accuracy: 0.9733 - auc: 0.9930

Epoch 00004: saving model to /home/jovyan/dataset/20epoch/model401_502/401_502.ckpt
Epoch 5/5
2699/2699 [==============================] - 727s 269ms/step - loss: 0.0903 - accuracy: 0.9753 - auc: 0.9934

Epoch 00005: saving model to /home/jovyan/dataset/20epoch/model4